In [ ]:
import shutil
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter, ImageOps, ImageDraw
import PIL
import glob
import pandas as pd
import numpy as np
import os
import random
import zipfile
import fnmatch

**YOLOv*** MUST have label data in a text (.txt) file with the same name as the img. Each label is in the following format, where x and y are the anchor (center) point of the object.
* <object-class-id> \<x> \<y> \<width> \<height>

# Data Management

# Master Dataframe

In [ ]:
# Copy and replace labels from source to home directory
# Labels

# Images
orig_images = glob.glob(r"Z:\__Organized_Directories_InProgress\Fish_annotations_based_on_YOLO\*\screened_with_fish\original_images\*\*.png")
orig_labels = glob.glob(r"Z:\__Organized_Directories_InProgress\Fish_annotations_based_on_YOLO\*\screened_with_fish\YOLO_bbox_coordinates\*.txt")
orig_obb = glob.glob(r"Z:\__Organized_Directories_InProgress\Fish_annotations_based_on_YOLO\*\screened_with_fish\YOLO_obb_coordinates\*.txt")
print(len(orig_images)) # 69831
print(len(orig_labels)) # 69831
print(len(orig_obb)) # 69831
# copy_img_lst_conv_jpg(orig_images_2022, dst)


In [ ]:
# All labels dataframe with potential avoidance flag
# df_20_23_all = generate_splits(img_pth_lst=orig_images, bbox_pths=orig_labels, mer_pths=orig_obb).return_merged()

# # Load master label dataframe
# master_lbl_df = pd.read_csv(r"Z:\__Organized_Directories_InProgress\Fish_annotations_based_on_YOLO\YOLO_coordinates_combined_2020_2023_bbox.csv", index_col=0)
# print(master_lbl_df.shape)  # (238028, 10) - with 2019  (231608, 11)

# # Add n_fish column to df_20_23_all
# df_20_23_alln = df_20_23_all.copy()
# nfpi = master_lbl_df.groupby("Filename").size().reset_index(name="n_fish")
# df_20_23_alln = df_20_23_alln.merge(nfpi, how="left")
# print(df_20_23_alln.year.value_counts())
# assert df_20_23_alln.shape[0] == df_20_23_all.shape[0]

# df_20_23_alln.to_csv(r"datasets\AUV_datasets\All_Goby_imgs_lbls_pths_n_fish.csv")
'''
year
2021    29228
2020    23339
2023     9052
2022     8213
Name: count, dtype: int64
'''

df_20_23_alln = pd.read_csv(r"..\datasets\AUV_datasets\All_Goby_imgs_lbls_pths_n_fish.csv", index_col=0)

In [ ]:
df_20_23_alln[df_20_23_alln.year==2022]

# Make master label dataframe

In [ ]:
# master_lbl_df = general.make_master_lbl_df(df_20_23_alln[["bbox_path", "year"]]) # (takes 45 min)
# master_lbl_df.to_csv(r"Z:\__Organized_Directories_InProgress\Fish_annotations_based_on_YOLO\YOLO_coordinates_combined_2020_2023_bbox.csv")
master_lbl_df = pd.read_csv(r"Z:\__Organized_Directories_InProgress\Fish_annotations_based_on_YOLO\YOLO_coordinates_combined_2020_2023_bbox.csv", index_col=0)
master_lbl_df

### Write a textfile of image list / label list for inference

In [ ]:
# general.write_list_txt(lbl_pth_txt, lbl_pths)
# general.write_list_txt(img_pth_txt, img_pths) 

#### Comparing Master Label Database to MER Database

#### QAQC task

In [ ]:
def copy_qaqc_images_plus_label():
    year = "2023"
    qaqc_df = df_20_23_alln[(df_20_23_alln.n_fish>10) & (df_20_23_alln.year==year)][["local_path", "image_id", "bbox_path", "year", "n_fish", "fish_type"]]
    # pth_lst = qaqc_df.local_path
    # general.copy_files_lst(pth_lst, dest, overwrite=False)
    print(qaqc_df.shape)
    dest = f"datasets\\alewife_qaqc\\{year}_lbl"
    if not os.path.exists(dest): os.mkdir(dest)
    for idx, row in qaqc_df.iterrows():
        local_path, image_id, bbox_path, year, n_fish, fish_type = row
        lbl_img = generate_annot_imgs.disp_lbl_bbox(local_path, bbox_path)
        lbl_img.save(os.path.join(dest,f"{image_id}_lbl.jpg")) 
def make_qaqc_excel(qaqc_df):    
    pth_lst = qaqc_df.local_path
    path = qaqc_df.image_id.apply(lambda x: f"{year}\\{x}" )
    qaqc_df["path"] = qaqc_df.image_id.apply(lambda x: f"{year}\\{x}.jpg")
    qaqc_df.iloc[:,1:].to_excel(os.path.join(dir,f"{year}_qaqc.xlsx"))

# Minimimum enclosing rectangles (MER)

In [ ]:
MER_cord = pd.read_csv(r"Z:\__Organized_Directories_InProgress\Fish_annotations_based_on_YOLO\_MERR_coordinates_all_years_id_fishtype_unfiltered.csv", low_memory=False, index_col=0)
fish_id_dropped = pd.read_csv(r"Z:\__Organized_Directories_InProgress\Fish_annotations_based_on_YOLO\fish_id_dropped.csv", index_col=0)

assert set(master_lbl_df.fish_id).difference(MER_cord.fish_id) == set()
print(MER_cord.shape) # (231609, 21)
MER_cord.head(3)

# 2019 - 2023 Run 11 Run 12
Run 11+ incorperates validation set for training

#### Train/Test/Valid Dataframes

In [ ]:
# Run 11/12
# s2019 = generate_splits().sample_df(df_22_23_all, "2019", 2000)
s2020 = generate_splits().sample_df(df_20_23_alln, "2020", 11000)
s2021 = generate_splits().sample_df(df_20_23_alln, "2021", 11000)
s2022 = generate_splits().sample_df(df_20_23_alln, "2022", 8212)
s2023 = generate_splits().sample_df(df_20_23_alln, "2023", 9052)
train_df, test_df, valid_df = generate_splits().chunk_split([s2020, s2021, s2022, s2023]) 
# train_df.to_csv(r"datasets\AUV_datasets\Runs\run12_png\train_df.csv")
# valid_df.to_csv(r"datasets\AUV_datasets\Runs\run12_png\valid_df.csv")
# test_df.to_csv(r"datasets\AUV_datasets\Runs\run12_png\test_df.csv")

# Run 11 Train (32960, 7) valid (4064, 7) test (4176, 7) 
# Train (31377, 7) valid (3888, 7) test (3968, 7) # run 12 jpg
# Train (31400, 8) valid (3888, 8) test (3976, 8) # run 12 png

In [ ]:
# run 13
s2020 = generate_splits().sample_df(df_20_23_alln, "2020", 9261)
s2021 = generate_splits().sample_df(df_20_23_alln, "2021", 5105)
s2022 = generate_splits().sample_df(df_20_23_alln, "2022", 1469)
s2023 = generate_splits().sample_df(df_20_23_alln, "2023", 976)
train_df, test_df, valid_df = generate_splits().chunk_split([s2020, s2021, s2022, s2023]) # Train (31401, 8) valid (3888, 8) test (3976, 8)
# Train (13435, 8) valid (1664, 8) test (1712, 8)
# train_df.to_csv(r"datasets\AUV_datasets\run13\train_df.csv")
# valid_df.to_csv(r"datasets\AUV_datasets\run13\valid_df.csv")
# test_df.to_csv(r"datasets\AUV_datasets\run13\test_df.csv")

In [ ]:
# run 14 - OBB with 2-class for alewife
s2020 = generate_splits().sample_df(df_20_23_alln, "2020", 11000)
s2021 = generate_splits().sample_df(df_20_23_alln, "2021", 11000)
s2022 = generate_splits().sample_df(df_20_23_alln, "2022", 8196)
s2023 = generate_splits().sample_df(df_20_23_alln, "2023", 9028)
train_df, test_df, valid_df = generate_splits().chunk_split([s2020, s2021, s2022, s2023], tr_frac=0.82, va_frac=0.1, year_splits = 8) 
alewife_id = list(set(MER_cord.image_id[MER_cord.cls==1]))
print(len(alewife_id))
print(len(train_df[train_df.image_id.isin(alewife_id)]))
print(len(valid_df[valid_df.image_id.isin(alewife_id)]))
print(len(test_df[test_df.image_id.isin(alewife_id)]))
'''
Train (32144, 8) valid (3932, 8) test (3148, 8)
62
35
10
13
'''
# the missing alewife images
all_df_id = pd.concat([train_df, valid_df, test_df]).image_id
aid = all_df_id[all_df_id.isin(alewife_id)]
# set(alewife_id).difference(aid)

# train_df.to_csv(r"datasets\AUV_datasets\run14\train_df.csv")
# valid_df.to_csv(r"datasets\AUV_datasets\run14\valid_df.csv")
# test_df.to_csv(r"datasets\AUV_datasets\run14\test_df.csv")

In [ ]:
# Create test set by year and copying to folder tree by year
root = "/caldera/projects/usgs/ecosystems/glsc/datasets/run11"
test_df_pth = "/caldera/projects/usgs/ecosystems/glsc/datasets/run11/test_df.csv"
'''
(4176, 7)
2020 (1320, 7)
2021 (1112, 7)
2022 (832, 7)
2023 (912, 7)
copying 912/912 2023 labels
'''
def make_set_by_year(test_df_pth, root):
    test_df = pd.read_csv(test_df_pth, index_col=0)
    years = test_df.year.unique()
    lens = []
    for year in years:
        test_dfyear = test_df[(test_df.year == year)]
        lens.append(len(test_dfyear))
        generate_splits.cpy_yr_tst(df=test_dfyear, year=year, root=root)   
    assert test_df.shape[0] == sum(lens)
root = r"datasets\AUV_datasets\Runs\run12_png"
test_df_pth = r"datasets\AUV_datasets\Runs\run12_png\test_df.csv"
# make_set_by_year(test_df_pth, root)